In [1]:
import geopandas as geopd
import pandas as pd
import folium
import rtree
import seaborn as sns
from shapely.geometry import shape
sns.set(rc={'figure.figsize':(10,10)})

In [2]:
building_data = geopd.read_file("https://raw.githubusercontent.com/NYCDOB/LocalLaw97/gh-pages/data/LL3320220609.json")
mapPLUTO = geopd.read_file("https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_mappluto_22v1_shp.zip")

In [3]:
emissions_trim = building_data.copy(deep = True)[["ID", "BBL_MapPLU", "EnergyStar", "LetterGrad", "geometry"]]
emissions_trim["BBL_MapPLU"] = emissions_trim.BBL_MapPLU.astype("str")

In [4]:
residential_buildings = mapPLUTO.query("UnitsRes > 0 & ResArea > 0 & ComArea == 0 & OfficeArea == 0 & RetailArea == 0")[["YearBuilt", "OwnerName", 'NumBldgs', 'NumFloors', 'UnitsRes', 'ResArea', 'Latitude', 'Longitude', "BBL", "YearAlter1", "YearAlter2"]]
residential_buildings["BBL_MapPLU"] = residential_buildings.copy(deep = True).BBL.astype("str").str.slice(0, 10)
residential_buildings = emissions_trim.merge(residential_buildings, how = "inner", on='BBL_MapPLU')
residential_buildings = geopd.GeoDataFrame(residential_buildings)

In [5]:
CDs = geopd.read_file("https://data.cityofnewyork.us/api/geospatial/yfnk-k7r4?method=export&format=GeoJSON")

In [6]:
residential_buildings["geometry"] = geopd.points_from_xy(residential_buildings["Longitude"], residential_buildings["Latitude"])
residential_buildings = geopd.GeoDataFrame(residential_buildings)

In [7]:
buildings_CDs = residential_buildings.overlay(CDs)

In [8]:
buildings_CDs["sqft_per_unit"] = buildings_CDs["ResArea"] / buildings_CDs["UnitsRes"]
buildings_CDs = buildings_CDs[buildings_CDs["YearBuilt"] > 1700]

CD_building_info = buildings_CDs.groupby("boro_cd").agg(avg_score = ("EnergyStar", "mean"),
                                     avg_sqft = ("sqft_per_unit", "mean"),
                                    count = ("BBL_MapPLU", "count"),
                                    avg_year = ("YearBuilt", "mean")).reset_index()

In [9]:
buildings_CDs.to_csv("data/buildings_CD.csv")
CD_building_info.to_csv("data/CD_building_info.csv")